<a href="https://colab.research.google.com/github/Girgis98/ParkingSlotDetectionModel/blob/main/ParkingDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config IPCompleter.greedy=True
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install torch-optimizer

     |████████████████████████████████| 72 kB 1.2 MB/s 


## Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch 
import torch_optimizer as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import scipy.io
import glob
from scipy.io import loadmat
import scipy
import pandas as pd
import io
import numpy as np
import cv2
import json
import os
import os.path
import cv2 as cv
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision import transforms, utils
import math
from skimage import io, transform
from collections import namedtuple
from enum import Enum
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from PIL import Image
from matplotlib import cm
import shutil
import codecs 
import timeit
import math
from tqdm import tqdm
import progressbar
import time
import pylab as pl
from IPython import display
import gc
import pickle
import random
import smtplib, ssl
from torch.profiler import profile, record_function, ProfilerActivity
from torch.utils.tensorboard import SummaryWriter

### Sending Notification Mail

In [ ]:
def send_notification():
  sender_email = "girgis.mamdouh@gmail.com"
  receiver_email = "girgis.mamdouh@gmail.com"
  SUBJECT = "Code finished"

  message = "Go continue running."
  message = 'Subject: {}\n\n{}'.format(SUBJECT, message)
  # Send email here

  port = 465  # For SSL
  password = "jaetmhgdilhannsw"

  # Create a secure SSL context
  context = ssl.create_default_context()

  with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, message)
def send_error():
  sender_email = "girgis.mamdouh@gmail.com"
  receiver_email = "girgis.mamdouh@gmail.com"
  SUBJECT = "Error"

  message = "Go rerun."
  message = 'Subject: {}\n\n{}'.format(SUBJECT, message)
  # Send email here

  port = 465  # For SSL
  password = "jaetmhgdilhannsw"

  # Create a secure SSL context
  context = ssl.create_default_context()

  with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, message)      

In [ ]:
device = torch.device("cuda") 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jul 26 06:01:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ConvNet class


In [ ]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    
    def __init__(self):
        dropout_prob = 0.0
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer7 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer8 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer10 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer91 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer101 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer11 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer13 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer121 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer131 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer14 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer15 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer16 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer151 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer161 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer152 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer162 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),nn.Dropout(p=dropout_prob))
        self.layer17 = nn.Sequential(
            nn.Conv2d(1024, 6, kernel_size=1, stride=1, padding=0),
            nn.Identity())
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer91(out)
        out = self.layer101(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer121(out)
        out = self.layer131(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer151(out)
        out = self.layer161(out)
        #print(out.shape)
        out = self.layer152(out)
        #print(out.shape)
        out = self.layer162(out)
        #print(out.shape)
        out = self.layer17(out)
        #print(out.shape)
        out = out.reshape(out.size(0), -1)
        #print(out.shape)
       
        return out

## Dataset class


In [ ]:
MarkingPoint = namedtuple('MarkingPoint', ['x', 'y', 'direction_x', 'direction_y', 'shape'])
class ParkingSlotDataset(Dataset):
    """Parking slot dataset."""

    def __init__(self, root, names_ls = []):

        super(ParkingSlotDataset, self).__init__()
        self.root = root
        self.sample_names = names_ls.copy()
        self.test_names = []
        self.all_samples = []
        self.image_transform = ToTensor()
        self.mode = 'train'
        if len(names_ls) == 0:
            accum = 0
            sub_dirs_num = (len([f for f in os.listdir(root)]))
            print("Number of Subdirectories is: ", sub_dirs_num, "\n")
            for i in range(sub_dirs_num):
                sub_root = f'{root}/{i}'
                print('\nfile is:', i)
                for file in os.listdir(sub_root):
                    if file.endswith(".json"):
                        self.sample_names.append(f'{i}/{os.path.splitext(file)[0]}')
                print('file len:', len(self.sample_names) - accum)
                accum = len(self.sample_names)
                print('total len:', len(self.sample_names))

    def split(self):
        total_num = len(self.sample_names)
        train_num = int(0.85 * total_num)

        random.Random(7).shuffle(self.sample_names)

        self.all_samples = self.sample_names.copy()
        self.sample_names = self.all_samples[0: train_num]
        self.test_names = self.all_samples[train_num:]

    def change_mode(self,mode):
      if mode == 'train' and self.mode == 'test':
        self.sample_names , self.test_names = self.test_names.copy() , self.sample_names.copy()  
        self.mode = 'train'
        print("Training...","\nTraining Dataset length is:",len(self.sample_names))

      elif mode == 'test' and self.mode == 'train':  
        self.sample_names , self.test_names = self.test_names.copy() , self.sample_names.copy()  
        self.mode = 'test'
        print("Testing...","\nTesting Dataset length is:",len(self.sample_names))



    def __getitem__(self, index):
        try:
            name = self.sample_names[index]
            image = cv.imread(f"{self.root}/{name}.jpg")
        except:
            print('error index is:', index)
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image2 = cv2.medianBlur(image, 9)
            smoothed = cv2.GaussianBlur(image2, (9, 9), 10)
            sharp = cv2.addWeighted(image2, 2, smoothed, -1, 0)
            bilat = cv2.bilateralFilter(sharp, 9, 75, 75)
            image = bilat

        except:
            print("error here", self.sample_names[index])
        image = self.image_transform(image)
        marking_points = []
        slots = []

        with open((f"{self.root}/{name}.json"), ) as file:
            if not isinstance(json.load(file)['marks'][0], list):
                with open((f"{self.root}/{name}.json"), ) as file:
                    marking_points.append(MarkingPoint(*json.load(file)['marks']))
            else:
                with open((f"{self.root}/{name}.json"), ) as file:
                    for label in np.array(json.load(file)['marks']):
                        marking_points.append(MarkingPoint(*label))

        with open((f"{self.root}/{name}.json"), ) as file:
            if (len(json.load(file)['slots']) == 0):
                pass
            else:
                with open((f"{self.root}/{name}.json"), ) as file:
                    if not isinstance(json.load(file)['slots'][0], list):
                        with open((f"{self.root}/{name}.json"), ) as file:
                            slots.append(Slot(*(json.load(file)['slots'])))
                    else:
                        with open((f"{self.root}/{name}.json"), ) as file:
                            for label in json.load(file)['slots']:
                                slots.append(Slot(*label))
        return {'image': image, 'marks': marking_points, 'slots': slots}

    def __len__(self):
        return len(self.sample_names)

    def __set__(self, index, value):
        # save file name then delete file thrn create another file with new data
        name = self.sample_names[index]
        in_image = value['image']
        in_image = in_image.permute(1, 2, 0)
        in_image = cv2.cvtColor(np.array(in_image), cv2.COLOR_RGB2BGR)
        in_image = cv.convertScaleAbs(in_image, alpha = (255.0))
        cv2.imwrite((f"{self.root}/{name}.jpg"), in_image)
        in_json = {}
        in_json = {'marks': value['marks'], 'slots': value['slots']}
        path = f"{self.root}/{name}.json"
        json.dump(in_json, codecs.open(path, 'w', encoding = 'utf-8'),
                  separators = (',', ':'), sort_keys = True)  ### this saves the array in .json format

## Rescale

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image =sample['image']
        marking_points = sample['marks']
        slots = sample['slots']

        h, w = image.shape[1:3]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        
        img = image.permute(1, 2, 0)
        img = transform.resize(img, (new_h, new_w))
        to_Tensor = ToTensor()
        img = to_Tensor(img)
        #img = img.permute(1, 2, 0)
        
        mult = ([(new_h / h) ,(new_w / w) ,(new_h / h) ,(new_w / w)])
        for i in range(len(marking_points)):
            iterable_mp = list(marking_points[i][0:4])
            for j in range(4):
                iterable_mp[j] = (iterable_mp[j] * mult[j])
            iterable_mp.append(marking_points[i][-1])    
            marking_points[i] = MarkingPoint(*iterable_mp)    
        return {'image': img,'marks': marking_points,'slots': slots}


##load

In [ ]:
def load(path,ls = []):
  try:
    park_dataset1 = ParkingSlotDataset(path,ls)
    return park_dataset1
  except:
    return load(path)  

In [ ]:
dataset_path = r"/content/drive/MyDrive/train_aug_all" # train_20_mix # train_two_hundred_uni # training_mix2
start = timeit.default_timer()
park_dataset = load(dataset_path)
stop = timeit.default_timer()
park_dataset.split()
print("\n\n")
print("Total Len is:",len(park_dataset.all_samples))
print("Train Len is:",len(park_dataset.sample_names))
print("Test Len is:",len(park_dataset.test_names))
print("Loading Time:",stop-start)
if park_dataset.sample_names.count("582/P_R_p2_img23_2076_aug_30") == 1:
  park_dataset.sample_names.remove("582/P_R_p2_img23_2076_aug_30")
send_notification()

In [ ]:
# park_dataset.change_mode("train")

## - Dataset Preprocessing

### + Image/Data rescaling

In [ ]:
# image_rescaler = Rescale((512, 512))
# for i in  range(len(park_dataset)):
#      if park_dataset[i]['image'].shape[1] != 512:
#          temp = image_rescaler(park_dataset[i])
#          park_dataset.__set__(i,temp)       

## Collate Function

In [ ]:
def collate_mod(data):

    d={}
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    #assert all('slots' in x for x in data)
    #assert all('marks' in x for x in data)
    #_, labels, lengths = (assert all('sentences' in x for x in batch))
    #print(labels,lengths)
    lengths_marks = []
    lengths_slots = []
    for i in range(len(data)):
      
      l1 =len(data[i]['marks'])
      l2 =len(data[i]['slots'])
      lengths_marks.append(l1)
      lengths_slots.append(l2)

    max_len_marks = max(lengths_marks)
    max_len_slots = max(lengths_slots)

    #n_ftrs = data[0][0].size(1)
    features_marks = torch.zeros(len(data),max_len_marks,5)
    features_slots = torch.zeros(len(data),max_len_slots,4)
    

    dict_im = torch.empty(len(data),3,512,512)
    for i in range(len(data)):
        j = len(data[i]['slots'])
        t =torch.zeros(max_len_slots-j,4)
        #print(torch.cat([(torch.Tensor(data[i]['slots'])), t]).shape)
        features_slots[i] = torch.cat([(torch.Tensor(data[i]['slots'])), t])


        k = len(data[i]['marks'])
        t =torch.zeros(max_len_marks - k,5)
        features_marks[i] = torch.cat([(torch.Tensor(data[i]['marks'])), t])

        dict_im[i]=data[i]['image']

    features_marks = features_marks.permute(1,2,0)
    features_slots =features_slots.permute(1,2,0)
    d= {'image' :dict_im, 'marks':features_marks,'slots':features_slots}
    #return features_marks.float(),features_slots.float()

   
    return d

## Grid locator Function

In [ ]:
def locate_grid_mult(x,y):
    if torch.sum(torch.less(x,0)):
        x[x<0] = 0
    if torch.sum(torch.less(y,0)):
        y[y<0] = 0
    if torch.sum(torch.greater(x,512)):
        x[x>512] = 512
    if torch.sum(torch.greater(y,512)):
        y[y>512] = 512
    grid_x = torch.divide(x,32).type(torch.int)
    grid_y = torch.divide(y,32).type(torch.int)
    grid_x[grid_x == 16] = 15
    grid_y[grid_y == 16] = 15
    
    return grid_x,grid_y

## Convert Marking Point to Tensor

In [ ]:
def marking_to_tensor(marking_points):
    output_tensor = torch.zeros((5,marking_points[0].shape[0])) #5_features,#_training
    for i in range(5):
        output_tensor[i] = marking_points[i]
    return output_tensor

## Complete 6x16x16 label Vector

In [ ]:
def calc_angle_2(p1,p2):
  y_diff = p2[1]- p1[1]
  x_diff = p2[0] - p1[0]
  theta = np.zeros_like(x_diff)
  theta = np.arctan2(y_diff,x_diff)
  return theta

In [ ]:
def complete_marking_vector_label_mult(training_examples):

    number_of_trainig_examples = training_examples['image'].shape[0]
    label_vector = torch.zeros((number_of_trainig_examples,6,16,16),dtype=torch.float64)

    label_vector[:,0,:,:] = label_vector[:,0,:,:] - 0.5
    
    for i in range(len(training_examples['marks'])):
        try:
          grid_x , grid_y = locate_grid_mult((training_examples['marks'][i][0]),
                                            (training_examples['marks'][i][1]))
          for j in range(number_of_trainig_examples):
            label_vector[j,1:,grid_x[j],grid_y[j]] = (marking_to_tensor(training_examples['marks'][i]))[:,j]
            label_vector[j,0,grid_x[j],grid_y[j]] = (0.5)  # confidence
        except:
          print("Problem at :" ,i,"\n\n",training_examples[i])
          with open(r"/content/drive/MyDrive/trouble_list.pkl", "wb") as fp:   #Pickling
            pickle.dump(training_examples, fp)
          with open(r"/content/drive/MyDrive/trouble_index.pkl", "wb") as fp:   #Pickling
            pickle.dump(i, fp)   

    label_vector[:,5,:,:] = (label_vector[:,5,:,:]) - 0.5 # shape
    
    x_old = torch.clone(label_vector[:,1,:,:])
    y_old = torch.clone(label_vector[:,2,:,:])
    x_val = (torch.clone(label_vector[:,1,:,:]) % 32) - 16
    y_val = (torch.clone(label_vector[:,2,:,:]) % 32) - 16
    dir_x = torch.clone(label_vector[:,3,:,:]) - (x_old - x_val)
    dir_y = torch.clone(label_vector[:,4,:,:]) - (y_old - y_val)

    marking_direction =  (calc_angle_2([x_val,y_val],[dir_x,dir_y]))
    cos_theta = torch.cos(marking_direction) #* 16 # just to be in same range (dont forget to divide) # cos
    sin_theta = torch.sin(marking_direction) #* 16 # just to be in same range (dont forget to divide) # sin
  
    label_vector[:,1,:,:] = ((label_vector[:,1,:,:] % 32) - 16)/32 # x
    label_vector[:,2,:,:] = ((label_vector[:,2,:,:] % 32) - 16)/32 # y
    label_vector[:,3,:,:] = torch.clone(cos_theta) * 0.5
    label_vector[:,4,:,:] = torch.clone(sin_theta) * 0.5 # check here law feh arkam btt7at ll false
    #label_vector = label_vector/10 

    return (label_vector)

## Image Visualizer 

In [ ]:
def image_visualizer(parking_image):
    plt.imshow(parking_image['image'].permute(1, 2, 0))
    for i in range(len(parking_image['marks'])):
        plt.plot([parking_image['marks'][i][0],parking_image['marks'][i][2]],[parking_image['marks'][i][1],parking_image['marks'][i][3]],'o')
    plt.show()

## Evaluation

In [ ]:
def eval(output, label):
    conf_flag = (label[:, 0, :, :] == 0.5)
    not_conf_flag = (label[:, 0, :, :] == -0.5)
    out = output.permute(0, 2, 3, 1)
    label_p = label.permute(0, 2, 3, 1)
    threhold_conf = 0.4
    threhold_coordinates = 0.3125
    therhold_direction_sin = 0.3
    therhold_direction_cos = 0.92

    true_negative = torch.sum(torch.bitwise_not(
        torch.bitwise_xor((out[not_conf_flag][:, 0] < threhold_conf), label_p[not_conf_flag][:, 0] == -0.5)).type(
        torch.float)).item()

    false_negative = torch.sum(
        (torch.bitwise_xor((out[not_conf_flag][:, 0] < threhold_conf), label_p[not_conf_flag][:, 0] == -0.5)).type(
            torch.float)).item()

    confidence_ones = torch.bitwise_not(
        torch.bitwise_xor((out[conf_flag][:, 0] >= threhold_conf), label_p[conf_flag][:, 0] == 0.5))
    x_val = (torch.abs(out[conf_flag][:, 1] - label_p[conf_flag][:, 1]) <= threhold_coordinates)
    y_val = (torch.abs(out[conf_flag][:, 2] - label_p[conf_flag][:, 2]) <= threhold_coordinates)
    x_dir_val = (torch.abs(out[conf_flag][:, 3] - label_p[conf_flag][:, 3]) <= therhold_direction_cos)
    y_dir_val = (torch.abs(out[conf_flag][:, 4] - label_p[conf_flag][:, 4]) <= therhold_direction_sin)
    shape_val = torch.bitwise_not(torch.bitwise_xor((out[conf_flag][:, 5] >= -0.05), label_p[conf_flag][:, 5] == 0.5))

    total_positive = (label_p[conf_flag][:, 0]).shape[0]

    c = torch.sum(confidence_ones).item()
    x = torch.sum(x_val).item()
    y = torch.sum(y_val).item()
    dx = torch.sum(x_dir_val).item()
    dy = torch.sum(y_dir_val).item()
    s = torch.sum(shape_val).item()

    # print("total positive:",total_positive)
    # print("conf:",c)
    # print("x:",x)
    # print("y:",y)
    # print("dx:",dx)
    # print("dy:",dy)
    # print("shape:",s)

    true_positive = torch.sum(torch.bitwise_and(confidence_ones, torch.bitwise_and
    (x_val, torch.bitwise_and(y_val, torch.bitwise_and(x_dir_val, torch.bitwise_and(y_dir_val, shape_val))))).type(
        torch.float)).item()

    false_positive = (total_positive - true_positive)

    accuracy = (true_negative + true_positive) / (true_negative + false_negative + true_positive + false_positive) if (
                                                                                                                                  true_negative + false_negative + true_positive + false_positive) != 0 else 0
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) != 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) != 0 else 0
    f1_score = 2 * (recall * precision) / (recall + precision) if (recall + precision) != 0 else 0
    print("TP:", true_positive, " TN:", true_negative, " FP:", false_positive, " FN:", false_negative)
    return accuracy, f1_score , precision , recall



## Compare weights

#### 350

In [ ]:
# model = ConvNet().to(device)
# model = model.to(device) 
# checkpoint = torch.load(r'/content/drive/MyDrive/350k_after_shape_cp1_v11') # 250k_after_shape_cp1_v5 # 100k_after_shape_cp1_v15 # 350k_after_shape_cp1_v3
# model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

### Train

In [ ]:
# park_dataset.change_mode("train")
# data_loader = DataLoader(park_dataset,
#                              batch_size=32, shuffle=False , num_workers=4,collate_fn=collate_mod,pin_memory=True)

In [ ]:
# model = model.eval()
# accum_f1 = 0
# accum_accu = 0
# accum_prec = 0
# accum_rec = 0
# ctr = 0
# with torch.no_grad():
#   for i_batch , train_batch in enumerate(data_loader):
#       images = train_batch['image'].to(device)
#       labels = complete_marking_vector_label_mult(train_batch).to(device)
#       outputs = model(images).to(device)
#       outputs = outputs.reshape((-1,6,16,16))
#       accuracy , evaluation , precision , recall = eval(outputs.float(), labels.float())
#       accum_f1 += evaluation
#       accum_accu += accuracy
#       accum_prec += precision
#       accum_rec += recall
#       ctr += 1
#       print("Current F1 score is:",(accum_f1/ctr)*100,"\nCurrent Accuracy is:",(accum_accu/ctr)*100,"\nCurrent Precision is:",(accum_prec/ctr)*100,"\nCurrent Recall is:",(accum_rec/ctr)*100,"\n\n")
# print("Final F1 score is:",(accum_f1/ctr)*100,"\nFinal Accuracy is:",(accum_accu/ctr)*100,"\nFinal Precision is:",(accum_prec/ctr)*100,"\nFinal Recall is:",(accum_rec/ctr)*100,"\n\n")

TP: 76.0  TN: 8088.0  FP: 14.0  FN: 14.0
Current F1 score is: 84.44444444444444 
Current Accuracy is: 99.658203125 
Current Precision is: 84.44444444444444 
Current Recall is: 84.44444444444444 


TP: 77.0  TN: 8079.0  FP: 22.0  FN: 14.0
Current F1 score is: 82.7485380116959 
Current Accuracy is: 99.609375 
Current Precision is: 81.11111111111111 
Current Recall is: 84.52991452991454 


TP: 72.0  TN: 8099.0  FP: 16.0  FN: 5.0
Current F1 score is: 84.25660109870637 
Current Accuracy is: 99.65413411458334 
Current Precision is: 81.34680134680136 
Current Recall is: 87.52210752210753 


TP: 77.0  TN: 8089.0  FP: 14.0  FN: 12.0
Current F1 score is: 84.58133971291866 
Current Accuracy is: 99.6612548828125 
Current Precision is: 82.16394716394717 
Current Recall is: 87.27079412472672 


TP: 72.0  TN: 8096.0  FP: 18.0  FN: 6.0
Current F1 score is: 84.80792891319207 
Current Accuracy is: 99.67041015625 
Current Precision is: 81.73115773115774 
Current Recall is: 88.27817376131983 


TP: 77.0  

KeyboardInterrupt: ignored

error index is: 620
error here 8/P_R_2081_aug_0
error index is: 524
error here 144/caro-par1006_aug_345


### Test

In [ ]:
# park_dataset.change_mode("test")
# data_loader = DataLoader(park_dataset,
#                              batch_size=32, shuffle=False , num_workers=4,collate_fn=collate_mod,pin_memory=True)

Testing... 
Testing Dataset length is: 54644


In [ ]:
# model = model.eval()
# accum_f1 = 0
# accum_accu = 0
# accum_prec = 0
# accum_rec = 0
# ctr = 0
# with torch.no_grad():
#   for i_batch , train_batch in enumerate(data_loader):
#       images = train_batch['image'].to(device)
#       labels = complete_marking_vector_label_mult(train_batch).to(device)
#       outputs = model(images).to(device)
#       outputs = outputs.reshape((-1,6,16,16))
#       accuracy , evaluation , precision , recall = eval(outputs.float(), labels.float())
#       accum_f1 += evaluation
#       accum_accu += accuracy
#       accum_prec += precision
#       accum_rec += recall
#       ctr += 1
#       print("Current F1 score is:",(accum_f1/ctr)*100,"\nCurrent Accuracy is:",(accum_accu/ctr)*100,"\nCurrent Precision is:",(accum_prec/ctr)*100,"\nCurrent Recall is:",(accum_rec/ctr)*100,"\n\n")
# print("Final F1 score is:",(accum_f1/ctr)*100,"\nFinal Accuracy is:",(accum_accu/ctr)*100,"\nFinal Precision is:",(accum_prec/ctr)*100,"\nFinal Recall is:",(accum_rec/ctr)*100,"\n\n")

TP: 76.0  TN: 8089.0  FP: 18.0  FN: 9.0
Current F1 score is: 84.91620111731844 
Current Accuracy is: 99.67041015625 
Current Precision is: 80.85106382978722 
Current Recall is: 89.41176470588236 


TP: 82.0  TN: 8085.0  FP: 16.0  FN: 9.0
Current F1 score is: 85.8443439449026 
Current Accuracy is: 99.6826171875 
Current Precision is: 82.26226660877116 
Current Recall is: 89.76082740788624 


TP: 69.0  TN: 8098.0  FP: 14.0  FN: 11.0
Current F1 score is: 85.45042152564059 
Current Accuracy is: 99.68668619791666 
Current Precision is: 82.5523544460081 
Current Recall is: 88.59055160525749 


TP: 77.0  TN: 8088.0  FP: 20.0  FN: 7.0
Current F1 score is: 85.35853437627465 
Current Accuracy is: 99.6826171875 
Current Precision is: 81.75962665924833 
Current Recall is: 89.35958037060978 


TP: 72.0  TN: 8090.0  FP: 20.0  FN: 10.0
Current F1 score is: 84.83855163895075 
Current Accuracy is: 99.6728515625 
Current Precision is: 81.05987524044215 
Current Recall is: 89.04863990624392 


TP: 72.0  

KeyboardInterrupt: ignored

error index is: 342
error here 711/P_R_p2_img58_0294_aug_165
error index is: 277
error here 10/P_R_2199_aug_195
error index is: 374
error here 53/P_R_1764_aug_135
error index is: 308
error here 361/ahm-par2299_aug_0


### 350 Variant

In [ ]:
# model = ConvNet().to(device)
# model = model.to(device) 
# checkpoint = torch.load(r'/content/drive/MyDrive/variant_350k_after_shape_cp1_v8') # 250k_after_shape_cp1_v5 # 100k_after_shape_cp1_v15 # 350k_after_shape_cp1_v3
# model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

### Train

In [ ]:
# park_dataset.change_mode("train")
# data_loader = DataLoader(park_dataset,
#                              batch_size=32, shuffle=False , num_workers=4,collate_fn=collate_mod,pin_memory=True)

Training... 
Training Dataset length is: 309643


In [ ]:
# model = model.eval()
# accum_f1 = 0
# accum_accu = 0
# accum_prec = 0
# accum_rec = 0
# ctr = 0
# with torch.no_grad():
#   for i_batch , train_batch in enumerate(data_loader):
#       images = train_batch['image'].to(device)
#       labels = complete_marking_vector_label_mult(train_batch).to(device)
#       outputs = model(images).to(device)
#       outputs = outputs.reshape((-1,6,16,16))
#       accuracy , evaluation , precision , recall = eval(outputs.float(), labels.float())
#       accum_f1 += evaluation
#       accum_accu += accuracy
#       accum_prec += precision
#       accum_rec += recall
#       ctr += 1
#       print("Current F1 score is:",(accum_f1/ctr)*100,"\nCurrent Accuracy is:",(accum_accu/ctr)*100,"\nCurrent Precision is:",(accum_prec/ctr)*100,"\nCurrent Recall is:",(accum_rec/ctr)*100,"\n\n")
# print("Final F1 score is:",(accum_f1/ctr)*100,"\nFinal Accuracy is:",(accum_accu/ctr)*100,"\nFinal Precision is:",(accum_prec/ctr)*100,"\nFinal Recall is:",(accum_rec/ctr)*100,"\n\n")

TP: 78.0  TN: 8088.0  FP: 12.0  FN: 14.0
Current F1 score is: 85.71428571428571 
Current Accuracy is: 99.6826171875 
Current Precision is: 86.66666666666667 
Current Recall is: 84.78260869565217 


TP: 77.0  TN: 8079.0  FP: 22.0  FN: 14.0
Current F1 score is: 83.38345864661653 
Current Accuracy is: 99.62158203125 
Current Precision is: 82.22222222222221 
Current Recall is: 84.69899665551839 


TP: 74.0  TN: 8100.0  FP: 14.0  FN: 4.0
Current F1 score is: 85.30784793308572 
Current Accuracy is: 99.67447916666666 
Current Precision is: 82.84511784511784 
Current Recall is: 88.0899293942772 


TP: 79.0  TN: 8090.0  FP: 12.0  FN: 11.0
Current F1 score is: 85.8040903697038 
Current Accuracy is: 99.6856689453125 
Current Precision is: 83.83713508713508 
Current Recall is: 88.01189149015235 


TP: 72.0  TN: 8096.0  FP: 18.0  FN: 6.0
Current F1 score is: 85.78612943862018 
Current Accuracy is: 99.68994140625 
Current Precision is: 83.06970806970806 
Current Recall is: 88.87105165366034 


TP: 7

KeyboardInterrupt: ignored

error index is: 1184
error here 68/P_R_0540_aug_45
error index is: 1091
error here 317/gig-par536_aug_210


In [ ]:
# park_dataset.change_mode("test")
# data_loader = DataLoader(park_dataset,
#                              batch_size=32, shuffle=False , num_workers=4,collate_fn=collate_mod,pin_memory=True)

Testing... 
Testing Dataset length is: 54644


In [ ]:
# model = model.eval()
# accum_f1 = 0
# accum_accu = 0
# accum_prec = 0
# accum_rec = 0
# ctr = 0
# with torch.no_grad():
#   for i_batch , train_batch in enumerate(data_loader):
#       images = train_batch['image'].to(device)
#       labels = complete_marking_vector_label_mult(train_batch).to(device)
#       outputs = model(images).to(device)
#       outputs = outputs.reshape((-1,6,16,16))
#       accuracy , evaluation , precision , recall = eval(outputs.float(), labels.float())
#       accum_f1 += evaluation
#       accum_accu += accuracy
#       accum_prec += precision
#       accum_rec += recall
#       ctr += 1
#       print("Current F1 score is:",(accum_f1/ctr)*100,"\nCurrent Accuracy is:",(accum_accu/ctr)*100,"\nCurrent Precision is:",(accum_prec/ctr)*100,"\nCurrent Recall is:",(accum_rec/ctr)*100,"\n\n")
# print("Final F1 score is:",(accum_f1/ctr)*100,"\nFinal Accuracy is:",(accum_accu/ctr)*100,"\nFinal Precision is:",(accum_prec/ctr)*100,"\nFinal Recall is:",(accum_rec/ctr)*100,"\n\n")

TP: 76.0  TN: 8088.0  FP: 18.0  FN: 10.0
Current F1 score is: 84.44444444444444 
Current Accuracy is: 99.658203125 
Current Precision is: 80.85106382978722 
Current Recall is: 88.37209302325581 


TP: 82.0  TN: 8086.0  FP: 16.0  FN: 8.0
Current F1 score is: 85.83924349881796 
Current Accuracy is: 99.6826171875 
Current Precision is: 82.26226660877116 
Current Recall is: 89.74160206718346 


TP: 71.0  TN: 8094.0  FP: 12.0  FN: 15.0
Current F1 score is: 85.2340518788964 
Current Accuracy is: 99.67854817708334 
Current Precision is: 83.3555672974137 
Current Recall is: 87.3471145564169 


TP: 75.0  TN: 8088.0  FP: 22.0  FN: 7.0
Current F1 score is: 84.87525957956336 
Current Accuracy is: 99.67041015625 
Current Precision is: 81.84657238027677 
Current Recall is: 88.37618957584925 


TP: 73.0  TN: 8089.0  FP: 19.0  FN: 11.0
Current F1 score is: 84.49111675455978 
Current Accuracy is: 99.6630859375 
Current Precision is: 81.34682312161272 
Current Recall is: 88.08190404163177 


TP: 72.0  T

KeyboardInterrupt: ignored

error index is: 742
error here 734/P_R_p2_img73_0846_aug_330
error index is: 604
error here 480/P_R_img8_2109_aug_330


## Experimental Loss

In [ ]:
sat= 10
initial =0.01
inflection=50

c=sat
a= c/initial - 1
b= np.log(a) / inflection
print(c,b,a)

def decay_fn(x):
  #exp_term = torch.mul(x,inflection)
  #exp = torch.exp(exp_term)
  #exp_1 = torch.add(exp,1)
  #inv = torch.pow(exp_1,-1)
  #y = torch.mul(inv,sat)

  exp_term = torch.mul(x,-1*b)
  exp = torch.exp(exp_term)
  exp_a = torch.mul(exp,a)
  exp_1 = torch.add(exp_a,1)
  inv = torch.pow(exp_1,-1)
  y = torch.mul(inv,c)
  #y = sat / (1+ initial * math.exp(-1*inflection*x[:]))
  return y

10 0.13813509557297107 999.0


## Loss Function

In [ ]:
def my_loss(output, label):
  conf_flag = (label[:,0,:,:] == 0.5) # change to 100
  not_conf_flag = (label[:,0,:,:] == -0.5)
  out = output.permute(0,2,3,1)
  la = label.permute(0,2,3,1)
  Loss1 = torch.mean((out[conf_flag] - la[conf_flag])**2)     # kolo
  Loss2 = torch.mean((out[not_conf_flag][:,0] - la[not_conf_flag][:,0])**2)  # confidence
  return torch.abs(Loss1) + torch.abs(Loss2) 

In [ ]:
data_loader = DataLoader(park_dataset,
                             batch_size=32, shuffle=True , num_workers=4,collate_fn=collate_mod,pin_memory=True)

## Training

In [ ]:
# Hyper parameters
num_epochs = 50000000 
learning_rate = 0.001
model = ConvNet().to(device)
model.to(device) 
optimizer = optim.RAdam(model.parameters(), lr=learning_rate)



checkpoint = torch.load(r'/content/drive/MyDrive/variant_350k_after_shape_cp1_v8') # 20
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
myloss = checkpoint['loss']
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min',factor=0.9, patience=50, cooldown=20, min_lr=0, verbose=False)
scheduler1 = (checkpoint['scheduler'])


model = model.train();


In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir=/content/drive/MyDrive/variant_350k_after_shape

In [ ]:
# scheduler1 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = 0.00000000001, max_lr=8e-4, step_size_up=6000, step_size_down=6000, mode='triangular', gamma=1, scale_fn=None, cycle_momentum=False, base_momentum=0.8, max_momentum=0.9, last_epoch=-1, verbose=False)
#scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min',factor=0.9, patience=40, cooldown=10, min_lr=0, verbose=False)

In [ ]:
n = 8
writer = SummaryWriter(f'/content/drive/MyDrive/variant_350k_after_shape/{n}') 
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/variant_350k_after_shape
total_step = len(data_loader)
graph_itrator = 0
graph_itrator_2 = 0
batch_loss = 0

actual_batch_size = 24 * 32 # 8192
BS = data_loader.batch_size
actual_batch_factor = actual_batch_size/BS
number_of_batches = math.ceil(len(park_dataset.sample_names)/BS)

model = model.train();

for epoch in range(num_epochs):
  for i_batch , train_batch in enumerate(data_loader):
    images = train_batch['image'].to(device)
    labels = complete_marking_vector_label_mult(train_batch).to(device)

    # Forward pass        
    outputs = model(images).to(device)
    outputs = outputs.reshape((-1,6,16,16))      
    myloss = my_loss(outputs.float(), labels.float())

    # Backward and optimize        
    myloss.backward()

    batch_loss += myloss.item()

    if graph_itrator % actual_batch_factor == 0:     
      optimizer.step()
      optimizer.zero_grad()
      if graph_itrator > 10:
        scheduler1.step(batch_loss)
        s  = scheduler1
        # if True: #(batch_loss/actual_batch_factor) > 0.017:
        #       scheduler.step()  
        #       s  = scheduler
        # else:  
        #   decreasing_scheduler.step(batch_loss)
        #   s  = decreasing_scheduler
       
        lrr = s.optimizer.param_groups[0]['lr']
        for g in optimizer.param_groups:
          g['lr'] = lrr

      for g in optimizer.param_groups:
        lr = g['lr']
    
    

    if graph_itrator % 120 == 0:  
      torch.save({
              'i_batch': i_batch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': myloss,
              'scheduler':scheduler1,                               # change here
              'batch_loss':batch_loss
              }, f'/content/drive/MyDrive/variant_350k_after_shape_cp1_v{n}') # final_15k_cp1_v1
    if graph_itrator % 240 == 0: 
      torch.save({
              'i_batch': i_batch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': myloss,
              'scheduler':scheduler1,                             # change here
              'batch_loss':batch_loss
              }, f'/content/drive/MyDrive/variant_350k_after_shape_cp2_v{n}')  # final_15k_cp2_v1
    
    if graph_itrator % actual_batch_factor == 0:
      graph_itrator_2 +=1 
      writer.add_scalar('training loss',
                            batch_loss/actual_batch_factor,
                            (graph_itrator_2 + 26.3e3 ))
      writer.add_scalar('lr',
                            lr,
                            (graph_itrator_2 + 26.3e3 ))
      batch_loss = 0
    graph_itrator +=1 


In [ ]:
park_dataset.sample_names.count("582/P_R_p2_img23_2076_aug_30")

0

In [ ]:
#prof.export_chrome_trace("trace.json")
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=100))

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=100))

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=100))

In [ ]:
prof.export_stacks("/tmp/profiler_stacks.txt", "self_cuda_time_total")

In [ ]:
torch.save({
              'i_batch': i_batch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': myloss
              }, r'/content/drive/MyDrive/model1_checkpoint_train_20_dropout_0.01')  

In [ ]:
while 1:
  time.sleep(1)

KeyboardInterrupt: ignored

## Prediction

In [ ]:
def mapper_vectorized(prediction_in):

    prediction = torch.clone(prediction_in) 
    prediction[:,1:3,:,:] = ( prediction[:,1:3,:,:] * 32) + 16
    values = torch.arange(0,512,32)
    xv, yv = torch.meshgrid([values, values])
    yv = yv.to(device) 
    xv = xv.to(device) 

    prediction[:,1,:,:] = prediction[:,1,:,:] + xv
    prediction[:,2,:,:] = prediction[:,2,:,:] + yv
    cos_value = prediction[:, 3, :, :] * 2
    sin_value = prediction[:, 4, :, :] * 2
    direction = torch.atan(sin_value / cos_value)
    x_val_mapped = prediction[:, 1, :, :]
    y_val_mapped = prediction[:, 2, :, :]
    x_dir = x_val_mapped + (40 * cos_value) 
    y_dir = y_val_mapped + (40 * sin_value) 

    prediction[:, 3, :, :] = x_dir
    prediction[:, 4, :, :] = y_dir
    prediction[:, 0, :, :] = (prediction[:, 0, :, :] + 0.5) * 100
    prediction[:, 5, :, :] = (prediction[:, 5, :, :] >= 0)

    return prediction

In [ ]:
def predict(image):
    out = model(image).to(device)
    out = out.reshape((-1,6,16,16))
    start = timeit.default_timer()
    out = mapper_vectorized(out)
    return out

## Visualizer 2

In [ ]:
def visualize_after_thres(image,prediction):
    pre = prediction
    plt.imshow(image.permute(1, 2, 0))
    for i in range(len(pre)):
        plt.plot([pre[i,1]],[pre[i][2]],'o')
        plt.plot([pre[i,3]],[pre[i][4]],'x')
    plt.show()

## Remove points lower than Thresthold 

In [ ]:
def get_predicted_points(prediction, thresh): # prediction (training,6,16,16)
    """Get marking points from one predicted feature map."""
    assert isinstance(prediction, torch.Tensor)
    if len(prediction.shape) == 3:
        prediction = prediction.reshape((1,6,16,16))
    num_training_examples = prediction.shape[0]
    predicted_points = []
    prediction = prediction.detach().cpu()
    
    index = (torch.greater_equal(prediction[:,0,:,:],thresh))
    
    prediction = prediction.permute(0,2,3,1)
    
    for i in range(num_training_examples):
        predicted_points.append(prediction[i,index[i,:,:]])

    predicted_points_copy = []
    for i in range(len(predicted_points)):
      predicted_points_copy.append(torch.clone(predicted_points[i]))

    for i in range(len(predicted_points)): # 3la el training examples
      for j in range(len(predicted_points[i])): # 3la el points
        if predicted_points[i][j][1] < 10 and  predicted_points[i][j][2] < 10:  # remove points with negative x , y values
           predicted_points_copy[i] = torch.cat([predicted_points_copy[i][0:j,:], predicted_points_copy[i][j+1:,:]])
    return (predicted_points_copy)

## Remove Row from Tensor

In [ ]:
def remove_row_ls(tens,row_index_ls):
    tens_copy = torch.clone(tens)
    ls = []
    for i in range(tens_copy.shape[0]):
        ls.append(i)
    ls = torch.tensor(ls)
    #index = []
    for i in range(len(row_index_ls)):
        index = ls[ls!=row_index_ls[i]]
        ls = index
    index = torch.tensor(index)
    return torch.index_select(tens_copy, 0, index)

## Non-max suppression for near points

In [ ]:
def non_maximum_suppression(pred_points): # pred_points (training,num_points,6) (list)
    """Perform non-maxmum suppression on marking points."""
    pred_copy = pred_points
    threshold_near = 40 # expermintal
    for k , tens in enumerate(pred_copy):
        #print(tens.shape[0])
        index_arr = []
        for i in range(tens.shape[0]):
            for j in range(i + 1, tens.shape[0]):
                i_x = tens[i][1]
                i_y = tens[i][2]
                j_x = tens[j][1]
                j_y = tens[j][2]
                abs_x = abs(j_x - i_x)
                abs_y = abs(j_y - i_y)
                abs_dis = math.sqrt(math.pow(abs_x,2) + math.pow(abs_y,2))
                #print(j,abs_dis)
                if abs_dis <= threshold_near:
                    #print("close")
                    idx = i if tens[i][0] < tens[j][0] else j
                    index_arr.append(idx)
        #print(i,index_arr)            
        #for i in range(len(index_arr)):   
        if(len(index_arr)!=0):
          pred_copy[k] = remove_row_ls(tens,index_arr)  
    return pred_copy        

## Calculate Distance between two points 

In [ ]:
def calc_point_squre_dist(point_a, point_b):
    """Calculate distance between two marking points."""
    x_a = point_a[1]
    y_a = point_a[2]
    x_b = point_b[1]
    y_b = point_b[2]

    distx = x_a - x_b
    disty = y_a - y_b
    dist = math.sqrt(math.pow(distx,2) + math.pow(disty,2))
    return dist

In [ ]:
i = 0 #22

In [ ]:
for x in range(0,len(park_dataset)):
  image_visualizer(park_dataset[x])
  print(x)
  #time.sleep(0.5)

In [ ]:
model  = model.eval()
start = timeit.default_timer()
predict_awal = predict(park_dataset[i]['image'].reshape((1,3,512,512)).to(device))
stop = timeit.default_timer()
print('Time: ', stop - start)  
predict_ba3d = get_predicted_points(predict_awal,80)
visualize_after_thres(park_dataset[i]['image'],predict_ba3d[0])
predict_ba3d[0]

In [ ]:
# Testing Single
image_path = r"/content/test_2.jpg"
input_image = cv.imread(image_path)
image_transform = ToTensor()
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
input_image = image_transform(input_image)
# input_image = input_image.permute(1,2,0)
# input_image = transform.resize(input_image, (512, 512))
# input_image = torch.tensor(input_image).permute(2,0,1)
model  = model.eval()
start = timeit.default_timer()
predict_awal = predict(input_image.reshape((1,3,512,512)).to(device))
stop = timeit.default_timer()
print('Time: ', stop - start)  
predict_ba3d = get_predicted_points(predict_awal,50)
predict2 = non_maximum_suppression(predict_ba3d)
visualize_after_thres(input_image,predict2[0])
print(predict2)

In [ ]:
# Testing data
root_path = r'/content/drive/MyDrive/2esmtany8arbev'
names = []
for file in os.listdir(root_path):
            if file.endswith(".jpg"):
                names.append(os.path.splitext(file)[0])
for i in range(0,len(names)):
  image_path = f"{root_path}/{names[i]}.jpg"
  input_image = cv.imread(image_path)
  input_image = cv2.medianBlur(input_image,5)
  smoothed = cv2.GaussianBlur(input_image, (9, 9), 10)
  input_image = cv2.addWeighted(input_image, 2, smoothed, -1, 5)
  image_transform = ToTensor()
  input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
  input_image = image_transform(input_image)
  #input_image = input_image.permute(1,2,0)
  #input_image = transform.resize(input_image, (512, 512))
  #input_image = torch.tensor(input_image).permute(2,0,1)
  model  = model.eval()
  start = timeit.default_timer()
  predict_awal = predict(input_image.reshape((1,3,512,512)).to(device))
  stop = timeit.default_timer()
  print('Time: ', stop - start)  
  predict_ba3d = get_predicted_points(predict_awal,70)
  predict2 = non_maximum_suppression(predict_ba3d)
  visualize_after_thres(input_image,predict2[0])
  print(i,predict2)

In [ ]:
park_dataset.change_mode("train")

Training... 
Training Dataset length is: 309643


In [ ]:
for j in range(2000,len(park_dataset)):
  model  = model.eval()
  start = timeit.default_timer()
  predict_awal = predict(park_dataset[j]['image'].reshape((1,3,512,512)).to(device))
  stop = timeit.default_timer()
  predict_ba3d = get_predicted_points(predict_awal,80)
  predict2 = non_maximum_suppression(predict_ba3d)
  visualize_after_thres(park_dataset[j]['image'],predict2[0])
  print(j,predict2[0])
  #time.sleep(0.5)

In [ ]:
park_dataset.change_mode("test")

Testing... 
Testing Dataset length is: 54644


In [ ]:
for j in range(3000,len(park_dataset)):
  model  = model.eval()
  start = timeit.default_timer()
  predict_awal = predict(park_dataset[j]['image'].reshape((1,3,512,512)).to(device))
  stop = timeit.default_timer()
  predict_ba3d = get_predicted_points(predict_awal,80)
  predict2 = non_maximum_suppression(predict_ba3d)
  visualize_after_thres(park_dataset[j]['image'],predict2[0])
  print(j,predict2[0])
  #time.sleep(0.5)

In [ ]:
predict_ba3d[0]

In [ ]:
predict2 = non_maximum_suppression(predict_ba3d)

In [ ]:
visualize_after_thres(park_dataset[i]['image'],predict2[0])

In [ ]:
predict_ba3d[0][:,1]

## Is there a third point that Passes through a line between these two points? 

In [ ]:
SLOT_SUPPRESSION_DOT_PRODUCT_THRESH = 0.8
def pass_through_third_point(marking_points, i, j):
    """See whether the line between two points pass through a third point."""
    x_1 = marking_points[i][1]
    y_1 = marking_points[i][2]
    x_2 = marking_points[j][1]
    y_2 = marking_points[j][2]
    for point_idx, tens in enumerate(marking_points):
        if point_idx == i or point_idx == j:
            continue
        x_0 = tens[1]
        y_0 = tens[2]
        vec1 = np.array([x_0 - x_1, y_0 - y_1])
        vec2 = np.array([x_2 - x_0, y_2 - y_0])
        vec1 = vec1 / np.linalg.norm(vec1)
        vec2 = vec2 / np.linalg.norm(vec2)
        if np.dot(vec1, vec2) > config.SLOT_SUPPRESSION_DOT_PRODUCT_THRESH:
            return True
    return False

In [ ]:
epsilon = 0.001  # check on number

# check if c between a,b
def isBetween(a, b, c):
    crossproduct = (c[2] - a[2]) * (b[1] - a[1]) - (c[1] - a[1]) * (b[2] - a[2])
    print ("crossproduct",crossproduct)
    # compare versus epsilon for floating point values, or != 0 if using integers 
    # sin 0 =0 , check theta >0 (not parallel)
    if abs(crossproduct) > epsilon:
        return False

    # cos 180 = -1, check that they are in the same direction
    dotproduct = (c[1] - a[1]) * (b[1] - a[1]) + (c[2] - a[2])*(b[2] - a[2])
    print ("dotproduct",dotproduct)
    if dotproduct < 0:
        return False

    squaredlengthba = (b[1] - a[1])*(b[1] - a[1]) + (b[2] - a[2])*(b[2] - a[2])
    print ("squaredlengthba",squaredlengthba)

    if dotproduct > squaredlengthba:
        return False

    return True


In [ ]:
a = [1,0,2]
b = [1,0,6]
c = [1,0,4]

test = isBetween (a,b,c)
print(test)

In [ ]:
decreased_points = non_maximum_suppression(outputs)
# pairing point 
for k , tens in enumerate(decreased_points):


## Calculate Direction(angle)

In [ ]:
def calc_angle(p1,p2):
  y_diff = p2[1]- p1[1]
  x_diff = p2[0] - p1[0]
  if x_diff == 0 and y_diff ==0 :
    return 0
  theta = math.atan(abs( y_diff/x_diff)) * (180/math.pi)
  
  
  if x_diff >=0 and y_diff >=0:
    return theta
  elif x_diff < 0 and y_diff >=0:
    return 180- theta
  elif x_diff < 0 and y_diff < 0:
    return  180 - theta
  else:
    return theta

## Calculate difference in direction

In [ ]:
def direction_diff(direction_a, direction_b):
    """Calculate the angle between two direction."""
    diff = abs(direction_a - direction_b)
    return diff if diff < math.pi else 2*math.pi - diff

## Determine Point Shape

In [ ]:
none = 0
l_down = 1
t_down = 2
t_middle = 3
t_up = 4
l_up = 5
BRIDGE_ANGLE_DIFF = 0.09757113548987695 + 0.1384059287593468
SEPARATOR_ANGLE_DIFF = 0.284967562063968 + 0.1384059287593468

def detemine_point_shape(point, vector):
    """Determine which category the point is in."""
    vec_direct = math.atan2(vector[1], vector[0])
    vec_direct_up = math.atan2(-vector[0], vector[1])
    vec_direct_down = math.atan2(vector[0], -vector[1])
    direction =  calc_angle ([point[1],point[2]],[point[3],point[4]])
    direction = (direction * math.pi)/180
    if point[5] < 0.5:
        if direction_diff(vec_direct, direction) < BRIDGE_ANGLE_DIFF:
            return t_middle
        if direction_diff(vec_direct_up, direction) < SEPARATOR_ANGLE_DIFF:
            return t_up
        if direction_diff(vec_direct_down, direction) < SEPARATOR_ANGLE_DIFF:
            return t_down
    else:
        if direction_diff(vec_direct, direction) < BRIDGE_ANGLE_DIFF:
            return l_down
        if direction_diff(vec_direct_up, direction) < SEPARATOR_ANGLE_DIFF:
            return l_up
    return none

## Pair Marking Points to form slots (can these two make a slot)

In [ ]:
def pair_marking_points(point_a, point_b):
    """See whether two marking points form a slot."""
    vector_ab = np.array([point_b[1] - point_a[1], point_b[2] - point_a[2]])
    vector_ab = vector_ab / np.linalg.norm(vector_ab)
    point_shape_a = detemine_point_shape(point_a, vector_ab)
    point_shape_b = detemine_point_shape(point_b, -vector_ab)
    if point_shape_a == 0 or point_shape_b == 0:
        return 0
    if point_shape_a == 3 and point_shape_b == 3:
        return 0
    if point_shape_a > 3 and point_shape_b > 3:
        return 0
    if point_shape_a < 3 and point_shape_b < 3:
        return 0
    if point_shape_a != 3:
        if point_shape_a > 3:
            return 1
        if point_shape_a < 3:
            return -1
    if point_shape_a == 3:
        if point_shape_b < 3:
            return 1
        if point_shape_b > 3:
            return -1

## Knowing Marking Points Get Slots

In [ ]:
perpend_min = 0.044771278151623496 *600 #26.862
perpend_max = 0.1099427457599304 *600   #65.96
parallel_min = 0.15057789144568634 *600 # 90.34
parallel_max = 350 # 266.69


def inference_slots(marking_points):
    """Inference slots based on marking points."""
    num_detected = marking_points.shape[0]
    perpen_parallel = 0  # perpendicular = 1 , parallel = 2
    
    slots ={}
    for i in range(num_detected - 1):
        for j in range(i + 1, num_detected):
            print (i,j)
            point_i = marking_points[i]
            point_j = marking_points[j]
            # Step 1: length filtration.
            distance = calc_point_squre_dist(point_i, point_j)
            print ("distance",distance)
            if not (perpend_min <= distance <= perpend_max
                    or parallel_min <= distance <= parallel_max):

                continue
                
            # Step 2: pass through filtration.
            if pass_through_third_point(marking_points, i, j):
                continue
            result = pair_marking_points(point_i, point_j)
            if  (perpend_min <= distance <= perpend_max):
                perpen_parallel = 1 # perpendiculer
            elif  (parallel_min <= distance <= parallel_max):
                perpen_parallel = 2 # parallel

            print("result",result)
            test1 ={'x1': marking_points[i][1],'y1': marking_points[i][2],'x2': marking_points[j][1],'y2': marking_points[j][2], 'type': perpen_parallel}
            test2 ={'x1': marking_points[j][1],'y1': marking_points[j][2],'x2': marking_points[i][1],'y2': marking_points[i][2], 'type': perpen_parallel}
            if result == 1:
               slots.update(test1)
            elif result == -1:
                slots.update(test2)
    return slots

In [ ]:
inference_slots(predict_ba3d[0])

In [ ]:
predict_ba3d[0].shape

In [ ]:
#monica
image = park_dataset [0]['image']
image.reshape((1,3,512,512))
predicted = predict(image)
y = get_predicted_points (predicted,threshold)
test = visualize_after_thres(image,y[0])


In [ ]:
test

In [ ]:
#outputs = process(outputs) # take marking points outputs and remove useless ones , non max suppression and do slot pairing

In [ ]:
# get slots
# outputs slots
# train
# evalute